<a href="https://colab.research.google.com/github/arunmergu/DeepLearning/blob/main/TranferLearning/Working_with_generic_images_and_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

In [2]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [3]:
ImageDataGenerator.flow_from_directory?

In [4]:
 !wget https://www.dropbox.com/s/uo4a18fhn7sl2d9/cat_dog_sub.zip

--2021-02-17 15:39:11--  https://www.dropbox.com/s/uo4a18fhn7sl2d9/cat_dog_sub.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/uo4a18fhn7sl2d9/cat_dog_sub.zip [following]
--2021-02-17 15:39:11--  https://www.dropbox.com/s/raw/uo4a18fhn7sl2d9/cat_dog_sub.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2fdc01e3df3705c2c2b8dcd01e.dl.dropboxusercontent.com/cd/0/inline/BJF1z2Mq-GMlcV00uGeI3h73s_kSodALZU4QZm9qAnavp4v-nRe_GaZdYJHmLRTkJRc7kwguPBO8frcgzCIZ0JEEvRJBE79x1OfMrJOvslNZp2W1kzdSCupylNDMNAU-o9k/file# [following]
--2021-02-17 15:39:12--  https://uc2fdc01e3df3705c2c2b8dcd01e.dl.dropboxusercontent.com/cd/0/inline/BJF1z2Mq-GMlcV00uGeI3h73s_kSodALZU4QZm9qAnavp4v-nRe_GaZdYJHmLRTkJRc7kwguPBO8frcgzCIZ0JEEvRJBE79x1Of

In [5]:
 !unzip cat_dog_sub.zip 

Streaming output truncated to the last 5000 lines.
  inflating: cat_dog_sub/test/dog/10363.jpg  
  inflating: __MACOSX/cat_dog_sub/test/dog/._10363.jpg  
  inflating: cat_dog_sub/test/dog/11069.jpg  
  inflating: __MACOSX/cat_dog_sub/test/dog/._11069.jpg  
  inflating: cat_dog_sub/test/dog/2156.jpg  
  inflating: __MACOSX/cat_dog_sub/test/dog/._2156.jpg  
  inflating: cat_dog_sub/test/dog/11727.jpg  
  inflating: __MACOSX/cat_dog_sub/test/dog/._11727.jpg  
  inflating: cat_dog_sub/test/dog/9995.jpg  
  inflating: __MACOSX/cat_dog_sub/test/dog/._9995.jpg  
  inflating: cat_dog_sub/test/dog/11082.jpg  
  inflating: __MACOSX/cat_dog_sub/test/dog/._11082.jpg  
  inflating: cat_dog_sub/test/dog/10388.jpg  
  inflating: __MACOSX/cat_dog_sub/test/dog/._10388.jpg  
  inflating: cat_dog_sub/test/dog/9942.jpg  
  inflating: __MACOSX/cat_dog_sub/test/dog/._9942.jpg  
  inflating: cat_dog_sub/test/dog/652.jpg  
  inflating: __MACOSX/cat_dog_sub/test/dog/._652.jpg  
  inflating: cat_dog_sub/test/do

In [7]:
 !rm -rf cat_dog_sub.zip

In [8]:
base_model=MobileNet(weights='imagenet',include_top=False)

17227776/17225924 [==============================] - 0s 0us/step


In [9]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, None, None, 

In [11]:
CLASSES = 2
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [12]:
for layer in base_model.layers:
    layer.trainable = False
      
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, None, None, 32)    0   

In [14]:
os.listdir('cat_dog_sub')

['.DS_Store', 'test', 'train']

In [15]:
 !rm -rf /content/cat_dog_sub/.DS_Store

In [16]:
WIDTH = 299
HEIGHT = 299
BATCH_SIZE = 32
TRAIN_DIR=r'/content/cat_dog_sub/train'
TEST_DIR=r'/content/cat_dog_sub/test'
# data prep
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
validation_generator = validation_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 1999 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [17]:
data=next(train_generator)

In [18]:
data[1].shape

(32, 2)

In [19]:
import tensorflow as tf

In [20]:
EPOCHS = 5
BATCH_SIZE = 32
STEPS_PER_EPOCH = train_generator.n//train_generator.batch_size
VALIDATION_STEPS=validation_generator.n//train_generator.batch_size

In [21]:
STEPS_PER_EPOCH,VALIDATION_STEPS

(62, 62)

In [22]:
model.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS,verbose=1)

Epoch 1/5
62/62 [==============================] - 103s 2s/step - loss: 0.4645 - accuracy: 0.7909 - val_loss: 0.1310 - val_accuracy: 0.9612
Epoch 2/5
62/62 [==============================] - 94s 2s/step - loss: 0.1667 - accuracy: 0.9381 - val_loss: 0.0961 - val_accuracy: 0.9642
Epoch 3/5
62/62 [==============================] - 94s 2s/step - loss: 0.1117 - accuracy: 0.9552 - val_loss: 0.0761 - val_accuracy: 0.9723
Epoch 4/5
62/62 [==============================] - 94s 2s/step - loss: 0.1069 - accuracy: 0.9589 - val_loss: 0.0662 - val_accuracy: 0.9768
Epoch 5/5
62/62 [==============================] - 94s 2s/step - loss: 0.1039 - accuracy: 0.9615 - val_loss: 0.0617 - val_accuracy: 0.9768


In [23]:
from tensorflow.keras.preprocessing import image

In [31]:
img = image.load_img(r'cat_dog_sub/test/cat/10011.jpg', target_size=(299,299))

In [32]:
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0]

In [33]:
pred

array([0.98332083, 0.0166791 ], dtype=float32)

In [35]:
train_generator.class_indices

{'cat': 0, 'dog': 1}